<a href="https://colab.research.google.com/github/vlasova-us/Python_public/blob/main/Parser_from_habr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Функция поиска на сайте habr. В качестве параметра функция принимает список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращает датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>


In [160]:
import time
from datetime import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import numpy as np

In [161]:
def get_all_links(query, pages):
  '''
Функция принимает на вход список из поискового запроса(ов) (query) для поиска на сайте https://habr.com , а также глубину поиска (количество поисковых страниц), которую необходимо вывести (pages).
Функция возвращает список из ссылок.

  '''
  url = 'https://habr.com/ru/search/'
  all_links = []
  links_list = []
  params = {'q': query, 'target_type': 'posts','order':'date'}

  for i in range(1, pages+1):
    if i == 1:
      req = requests.get(url, params)
    else: req = requests.get((str(url) + str('page')+str(i)+'/'), params)

    soup = BeautifulSoup (req.text)

    time.sleep(0.33)

    refs = soup.find_all('article', class_='tm-articles-list__item')

    for ref in refs:
      all_links.append(ref.find('a', 'tm-title__link').get('href'))
      all_full_links = list(map(lambda x: 'https://habr.com' + x, all_links))   #Ссылки
      links_list += all_full_links
  return set(links_list)

In [162]:
def search_habr (all_links):
  '''
Функция принимает на вход список из ссылок, полученных функцией get_all_links, от поискового запроса(ов) (query) для поиска на сайте https://habr.com
Функция возвращает DataFrame из столбцов:
- Дата опубликования статьи в datatime формате (колонка date)
- Заголовок статьи (колонка title)
- Ссылка на статью (колонка link)
- Полный текст статьи (колонка text)
- Количество лайков: общее кол-во, позитивные, негативные (колонка like)
  '''
  habr_articles_df=pd.DataFrame()
  for link in all_links:
    soup = BeautifulSoup(requests.get(link).text)
    time.sleep(0.33)

    date = pd.to_datetime (soup.find ('span', class_='tm-article-datetime-published').find('time').get('datetime')) #Дата в формате Datetime

    if soup.find ('h1', class_='tm-title tm-title_h1'): #Заголовок.
     title = soup.find ('h1', class_='tm-title tm-title_h1').find('span').text
    else: title = np.nan

    if soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-2'): #Полный текст
      text = soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-2').text.strip().replace('\n', '')
    elif soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-1'):
       text = soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-1').text.strip().replace('\n', '')
    else: text = np.nan

    row = {'date': date, 'title': title, 'link': link, 'text': text}
    habr_articles_df = pd.concat ([habr_articles_df, pd.DataFrame([row]) ] ) #  Объединение в DataFrame всех строк
    habr_articles_df ['date'] = pd.to_datetime(habr_articles_df ['date']).apply(lambda x: x.date()) #Убираем TimeZone, чтобы прошла выгрузка в Excel



  return habr_articles_df.reset_index (drop = True)


In [163]:
def main ():
  query = input ("Введите запрос: ")
  pages = int(input ("Введите количество страниц поиска: ") )
  all_links = get_all_links(query, pages)
  habr_ = search_habr (all_links)
  display(habr_)
  # habr_.to_excel('./result_search.xlsx')


In [164]:
main()

Введите запрос: dwh
Введите количество страниц поиска: 5


,date,title,link,text
0,2023-03-31,Партицирование в Greenplum 7: что нового,https://habr.com/ru/companies/southbridge/arti...,"Greenplum 7 — первая версия СУБД, совместимая ..."
1,2023-06-28,Разработка — всё? Действительно ли нас всех за...,https://habr.com/ru/companies/avito/articles/7...,Привет! Меня зовут Александр Пряхин. Я руковож...
2,2023-07-19,Каталог данных на примере DataHub. Часть I,https://habr.com/ru/articles/748350/,В современных компаниях генерируемые и собирае...
3,2023-06-21,Кто управляет информацией — тот владеет миром:...,https://habr.com/ru/articles/742910/,"Конечно, про ""владение миром"" - это перефразир..."
4,2022-12-20,Курсы для ИТ-специалистов от компаний: как они...,https://habr.com/ru/companies/tinkoff/articles...,Есть много способов получить ИТ-специальность ...
...,...,...,...,...
95,2023-07-20,Тестировщики тоже продакты: как убедить команд...,https://habr.com/ru/companies/hh/articles/748840/,"Всем привет! Меня зовут Максим, я работаю тест..."
96,2022-11-03,Как в Тинькофф создавали Data Catalog,https://habr.com/ru/companies/oleg-bunin/artic...,В чем главная задача аналитика? Думать головой...
97,2023-02-08,Платформа данных в Леруа Мерлен — как мы побед...,https://habr.com/ru/companies/leroy_merlin/art...,"Всем привет! Меня зовут Александр Токарев, я т..."
98,2022-12-15,Оптимизация данных в MS SQL,https://habr.com/ru/articles/705656/,Основная статья Взаимодействие DWH Oracle и MS...
